In [ ]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')

def f(value, plus=False):
    """ formata número 1.234,56 """
    if value is None: return None
    value = value if type(value) == int else float(value)
    r = format(value, ",").replace(".","!").replace(",",".").replace("!",",")
    return f"+{r}" if plus and value > 0 else r

SAVE_IMAGES = True


In [ ]:
VACCINES = {
    'COM': 'Pfizer',
    'AZ': 'AstraZeneca',
    'MOD': 'Moderna',
    'JANSS': 'Janssen',
}

IDADES = {
  # total 10_347_892  # 2020 = 10_298_253  # 2019 = 10_298_253
  '0_17':  1_736_215,  # 1_707_318,  # 16,51%  1_701_687,
  '18_24':   781_175,  #   781_176,  #  7,55%    781_176,   18-24: 29,7% 24,2% 46,1% - 3,48%
  '25_49': 3_331_561,  # 3_331_561,  # 32,22%  3_331_561,   25-49: 69,7% 13,7% 16,6% - 5.34%
  '50_64': 2_174_181,  # 2_174_181,  # 21,03%  2_174_181,   50-64: 90,2%  6,2%  3,7% - 0.007%
  '65_79': 1_643_170,  # 1_659_217,  # 16,05%  1_628_058,   65-79: 97,2%  2,8%  0,0%
  '80+':     681_590,  #   686_755,  #  6,64%    681_590,   80+  : 96,7%  3,3%  0,0%
}
TOTAL_IDADES = sum( IDADES.values() )
# print( list(map(lambda x: f"{x[0]} {x[1]} {round(x[1] / TOTAL_IDADES * 100, 2)}", IDADES.items() ) ))
print( (7.55 * 46.1 / 100, 32.22 * 16.6 / 100, 0.2103 * 3.7 / 100, ))

POP_PT = 10_347_892  # 2020 = 10_298_253  # 2019 = 10_295_909

def vaccine_name(vaccine):
  return VACCINES.get(vaccine, vaccine)


In [ ]:
# https://qap.ecdc.europa.eu/public/extensions/COVID-19/vaccine-tracker.html#distribution-tab
# https://www.ecdc.europa.eu/en/publications-data/data-covid-19-vaccination-eu-eea
!curl -s --output '/tmp/ecdc_vacinas.csv' "https://opendata.ecdc.europa.eu/covid19/vaccine_tracker/csv/data.csv"
!tail -1 '/tmp/ecdc_vacinas.csv'


In [ ]:
data = pd.read_csv('/tmp/ecdc_vacinas.csv') # , parse_dates=['data'], index_col='data', dayfirst=True)
data = data.sort_values(['YearWeekISO', 'ReportingCountry'])
data['date'] = data['YearWeekISO'].apply(lambda x: datetime.datetime.strptime(f"{x}-6", "%Y-W%W-%w"))
data.loc[ data['YearWeekISO'] == '2020-W52', 'date' ] += datetime.timedelta(days=-7)
data.loc[ data['YearWeekISO'] == '2020-W53', 'date' ] += datetime.timedelta(days=-7)
# data.set_index('date', inplace=True)


In [ ]:
data.tail(1)


In [ ]:
data['Vaccine'].unique()


In [ ]:
dfpt = data[ data['ReportingCountry'] == 'PT' ].copy()
dfpt.drop('ReportingCountry', axis=1, inplace=True)
if dfpt['FirstDoseRefused'].notnull().sum() == 0:
  dfpt.drop('FirstDoseRefused', axis=1, inplace=True)

# dfpt.describe()
# PT may have one less week that other countries
dfpt.tail()


In [ ]:
# Vacinas em Portugal
# ['COM', 'MOD', 'AZ', 'JANSS']
list(dfpt['Vaccine'].unique())


In [ ]:
df = dfpt.copy()
dft = df[ (df['TargetGroup'] == 'ALL') & (df['Region'] == 'PT') ]
dft = dft.drop(['Population', 'Denominator'], axis=1)
dft # .tail()


In [ ]:
# Primeiras doses AstraZeneca
dft[dft.Vaccine == 'AZ'][['FirstDose']]


In [ ]:
dft[dft.Vaccine == 'AZ'][['SecondDose']].tail(14)


In [ ]:
janssen = dft[dft.Vaccine == 'JANSS'].copy()
janssen['CUMSUM'] = janssen['FirstDose'].cumsum()
janssen


In [ ]:
dft[dft.Vaccine == 'JANSS'][['FirstDose', 'NumberDosesReceived']].sum()


In [ ]:
dft[dft.Vaccine == 'JANSS'][['FirstDose', 'NumberDosesReceived']].cumsum()


In [ ]:
# copy Janssen first dose to second dose
dft.loc[dft.Vaccine == 'JANSS', 'SecondDose'] = dft[dft.Vaccine == 'JANSS']['FirstDose']

dftt = dft.copy()

dft[dft.Vaccine == 'JANSS']


In [ ]:
# rows duplicadas?
dft[dft.Vaccine == 'AZ']


In [ ]:
# Ajusta a data da primeira dose para a data prevista para a
# segunda dose, conforme a marca e as regras na altura
dft = dftt.copy()
rows = []
ajuste_AZ, ajuste = 0, 0
seen = set()
for k, row in dft.iterrows():
    k = row['date']
    v, d1 = row.Vaccine, row.FirstDose
    if v == 'COM':
        # Cominarty / Pfizer era 21 dias até Março, depois 28
        k2 = k + datetime.timedelta(days=(28 if k >= datetime.datetime(2021, 3, 1) else 21))
    elif v == 'JANSS':
        # Janssen é dose única
        k2 = k  # 1st is 2nd
    elif v == 'AZ':
        # AstraZeneca era 12 semanas até final Junho
        # https://twitter.com/dgsaude/status/1411056606792622081?s=11
        # "até 25 abril -> até 4 julho" -> 2021-04-24 + 10 semanas = 2021-07-03
        # "de 26 abril a 16 maio -> de 5 a 11 julho" -> 2021-05-01 / 08 / 15 + 8 semanas = 2021-07-10
        # "de 17 maio a 23 maio -> a partir 12 julho" -> 2021-05-22 + 8 semanas = 2021-07-17
        k2 = k + datetime.timedelta(weeks=(8 if k >= datetime.datetime(2021, 5, 22) else 12))

        # 2021-05-23 + 8 = 2021-07-18 (dia seguinte é 24)
        if k2 >= datetime.datetime(2021, 6, 26) : # and k2 <= datetime.datetime(2021, 7, 31):
            ymd = k2.strftime("%Y-%m-%d")
            if ymd in seen:
                continue
            seen.add(ymd)
            try:
                second_dose = int(dft[(dft.date == ymd) & (dft.Vaccine == 'AZ')].SecondDose)+0
                ajuste = second_dose - d1
                ajuste_AZ += ajuste
                print(f"ajuste {ymd} depois second_dose={second_dose} previsao={d1} ajuste={ajuste} resto={ajuste_AZ}")
                d1 += ajuste
            except TypeError: # can't convert empty series to int or something
                ajuste = -1 * int(min(d1, ajuste_AZ) / 2)
                ajuste_AZ += ajuste
                print(f"ajuste {k2.strftime('%Y-%m-%d')} depois previsao={d1} ajuste={ajuste} resto={ajuste_AZ}")
                d1 += ajuste

    else:
        k2 = k + datetime.timedelta(days=28)
    rows.append([k2, v, d1])
# AJUSTE 21-28 dias
rows.append([datetime.datetime.strptime('2021-03-27', '%Y-%m-%d'), 'COM', 20_000])

df2 = pd.DataFrame(rows, columns=["d", "Vaccine", "Prediction"])
df2.sort_values("d", inplace=True)
# join overlapping AZ dates
df2['Prediction'] = df2.groupby(['d', 'Vaccine'])['Prediction'].transform('sum')
df2.drop_duplicates(inplace=True)
#print(df2)

df2.set_index("d", inplace=True)
df_prev = df2.copy()
#df2[df2.Vaccine=='AZ'].tail(14)


In [ ]:
df2 = df_prev.copy()
dft = dftt.copy()
dft.set_index('date', inplace=True)

maxY = 300_000
for v in [1,2]:
    if v == 2:
        df2 = df_prev.copy()
        #print(df2[df2.Vaccine=='AZ'].tail(10))
        df2['Prediction'] = df2.groupby(['d', 'Vaccine'])['Prediction'].transform('sum')
        df2.drop_duplicates(inplace=True)
        #print(df2[df2.Vaccine=='AZ'].tail(10))
        dft = dftt.copy()
        dft.set_index('date', inplace=True)
    v = 'AZ'

    fig, axes = plt.subplots(figsize=(15,5))
    k, kk='Prediction', 'Previsão'
    axes.plot(df2[ df2.Vaccine == v ][k], label=f"{VACCINES[v]} {kk}", lw=4, marker='o', markersize=8, alpha=0.5) # , color=C[v][0])
    k, kk='SecondDose', 'Vacinados'
    k='SecondDose' if v != "JANSS" else 'FirstDose'
    axes.plot(dft[ dft.Vaccine == v ][k], label=f"{VACCINES[v]} {k}", lw=4, marker='s', markersize=8, linestyle="--", alpha=0.5) # , color=C[v][1], alpha=0.5)

    plt.xticks(list(reversed(list(reversed(list(set(list(dft.index)+list(df2.index)))))[0::1])), rotation=90)
    axes.axvline(linewidth=2, ls="--", color='white', alpha=0.75, x=datetime.datetime.today())
    axes.axvline(linewidth=2, ls="--", color='white', alpha=0.75, x=datetime.datetime.today()+datetime.timedelta(days=28))
    plt.ylim(0, maxY)
    axes.legend(loc="upper left")
    plt.tight_layout()
    if SAVE_IMAGES: plt.savefig(f"../../temp/vacinas_ecdc_previsao_{v}.png")
    plt.show()
    print()


In [ ]:
# ['2021-04-03', '2021-04-10', '2021-04-17', '2021-04-24', '2021-05-01', '2021-05-08', '2021-05-15', '2021-05-22', '2021-05-29', '2021-06-05', '2021-06-12', '2021-06-19', '2021-06-26']
# print( list( map(lambda x: str(x)[:10], dfpt[ dfpt['date'] > datetime.datetime.strptime('2021-04-01', '%Y-%m-%d') ]['date'].unique()) ) )
# 2021-07-03 2021-07-10 2021-07-17 2021-07-24 2021-07-31 2021-08-07 2021-08-14 2021-08-21 2021-08-28 2021-09-04
#for i in range(0, 10):
#    print(str( datetime.datetime.strptime('2021-07-03', '%Y-%m-%d') + datetime.timedelta(weeks=i) )[:10])

# 2021-04-24 + 10 semanas = 2021-07-03 - "até 25 abril -> até 4 julho"
# 2021-05-01 / 08 / 15 + 8 semanas = 2021-07-10 - "de 26 abril a 16 maio -> de 5 a 11 julho"
# 2021-05-22 + 8 semanas = 2021-07-17 - "de 17 maio a 23 maio -> a partir 12 julho"
#print(str( datetime.datetime.strptime('2021-05-22', '%Y-%m-%d') + datetime.timedelta(weeks= +8) )[:10])


In [ ]:
df2 = df_prev.copy()
dft = dftt.copy()
dft.set_index('date', inplace=True)

CC = "#00876c,#57a18b,#8cbcac,#bed6ce,#f1f1f1,#f1c6c6,#ec9c9d,#ff7700,#0077ff,#e27076,#d43d51".split(",")
C = {
    'COM': CC[0:2],
    'AZ': CC[2:4],
    'MOD': CC[4:6],
    'JANSS': CC[6:8],
    'T': CC[8:10],
}

fig, axes = plt.subplots(figsize=(15,5))
k, kk='Prediction', 'Previsão'
for v in VACCINES:
    axes.plot(df2[ df2.Vaccine == v ][k], label=f"{VACCINES[v]} {kk}", lw=2, marker='o', linestyle="--", markersize=4)#, alpha=0.5)#, color=C[v][0])
axes.plot(df2.groupby(df2.index).sum()[k], label=f"Total {kk}", lw=4, marker='o', linestyle="--", markersize=8)#, alpha=0.5)#, color=C['T'][0])

k, kk='SecondDose', 'Vacinados'
for v in VACCINES:
    axes.plot(dft[ dft.Vaccine == v ][k], label=f"{VACCINES[v]} {kk}", lw=2, marker='s', markersize=4)#, alpha=0.5)#, color=C[v][1])
axes.plot(dft.groupby(dft.index).sum()[k], label=f"Total {kk}", lw=4, marker='s', markersize=9)#, alpha=0.5)#, color=C['T'][1])

plt.xticks(list(reversed(list(reversed(list(set(list(dft.index)+list(df2.index)))))[0::1])), rotation=90)
axes.axvline(linewidth=2, ls="--", color='white', alpha=0.75, x=datetime.datetime.today())
axes.axvline(linewidth=2, ls="--", color='white', alpha=0.75, x=datetime.datetime.today()+datetime.timedelta(days=28))
axes.legend(loc="upper left")
plt.ylim(0)
plt.tight_layout()
if SAVE_IMAGES: plt.savefig("../../temp/vacinas_ecdc_previsao.png")
plt.show()


In [ ]:
maxY = 0
for v in VACCINES:
    x = dft[ dft.Vaccine == v ].max()
    maxY = max(maxY, x['FirstDose'])
    maxY = max(maxY, x['SecondDose'])
print(maxY)
maxY = maxY * 1.1

In [ ]:
fig, axes = plt.subplots(figsize=(15,5))
k, kk='Prediction', 'Previsão'
axes.plot(df2.groupby(df2.index).sum()[k], label=f"Total {kk}", lw=4, marker='o', markersize=8, alpha=0.5) # , color=C[v][0])
k, kk='SecondDose', 'Vacinados'
axes.plot(dft.groupby(dft.index).sum()[k], label=f"Total {kk}", lw=4, marker='s', markersize=8, linestyle="--", alpha=0.5) # , color=C[v][1], alpha=0.5)

plt.xticks(list(reversed(list(reversed(list(set(list(dft.index)+list(df2.index)))))[0::1])), rotation=90)
axes.axvline(linewidth=2, ls="--", color='white', alpha=0.75, x=datetime.datetime.today())
axes.axvline(linewidth=2, ls="--", color='white', alpha=0.75, x=datetime.datetime.today()+datetime.timedelta(days=28))
#plt.ylim(0, maxY)
plt.ylim(0)
axes.legend(loc="upper left")
plt.tight_layout()
if SAVE_IMAGES: plt.savefig(f"../../temp/vacinas_ecdc_previsao_total.png")
plt.show()
print()

for v in VACCINES:
    fig, axes = plt.subplots(figsize=(15,5))
    k, kk='Prediction', 'Previsão'
    axes.plot(df2[ df2.Vaccine == v ][k], label=f"{VACCINES[v]} {kk}", lw=4, marker='o', markersize=8, alpha=0.5) # , color=C[v][0])
    k, kk='SecondDose', 'Vacinados'
    k='SecondDose' if v != "JANSS" else 'FirstDose'
    axes.plot(dft[ dft.Vaccine == v ][k], label=f"{VACCINES[v]} {k}", lw=4, marker='s', markersize=8, linestyle="--", alpha=0.5) # , color=C[v][1], alpha=0.5)

    plt.xticks(list(reversed(list(reversed(list(set(list(dft.index)+list(df2.index)))))[0::1])), rotation=90)
    axes.axvline(linewidth=2, ls="--", color='white', alpha=0.75, x=datetime.datetime.today())
    axes.axvline(linewidth=2, ls="--", color='white', alpha=0.75, x=datetime.datetime.today()+datetime.timedelta(days=28))
    plt.ylim(0, maxY)
    axes.legend(loc="upper left")
    plt.tight_layout()
    if SAVE_IMAGES: plt.savefig(f"../../temp/vacinas_ecdc_previsao_{v}.png")
    plt.show()
    print()


In [ ]:
fig, axes = plt.subplots(figsize=(15,5))
k, kk='Prediction', 'Previsão'
axes.plot(df2.groupby(df2.index).sum().cumsum()[k], label=f"Total {kk}", lw=4, marker='o', markersize=8, alpha=0.5) # , color=C[v][0])
#print(df2.groupby(df2.index).sum().cumsum()[k])
k, kk='SecondDose', 'Vacinados'
axes.plot(dft.groupby(dft.index).sum().cumsum()[k], label=f"Total {kk}", lw=4, marker='s', markersize=8, linestyle="--", alpha=0.5) # , color=C[v][1], alpha=0.5)

import math
millions = math.ceil(df2.groupby(df2.index).sum().cumsum()[-1:]['Prediction'] / 1_000_000.0)
for i in range(1, max(8, millions) + 1):
    plt.axhline(y=i*1000000, ls="--", alpha=0.75)

plt.axhline(y=POP_PT * 0.7, ls="--", alpha=0.75, lw=2, color="green")
last_val = dft.groupby(dft.index).sum().cumsum()['SecondDose'].max()
plt.axhline(y=last_val, ls="--", alpha=0.75, color="yellow")

plt.xticks(list(reversed(list(reversed(list(set(list(dft.index)+list(df2.index))))))), rotation=90)
axes.axvline(linewidth=2, ls="--", color='white', alpha=0.75, x=datetime.datetime.today())
axes.axvline(linewidth=2, ls="--", color='white', alpha=0.75, x=datetime.datetime.today()+datetime.timedelta(days=28))
axes.legend(loc="upper left")
plt.ylim(0)
plt.tight_layout()
if SAVE_IMAGES: plt.savefig(f"../../temp/vacinas_ecdc_previsao_accumulado.png")
plt.show()


In [ ]:
# Recebidas
dfr = dft[ dft['NumberDosesReceived'] > 0 ].copy()
dfr['Resto'] = dfr['NumberDosesReceived'] - dfr['FirstDose'] - dfr['SecondDose']
dfr = dfr.groupby(['YearWeekISO']).sum()
axes = dfr[['FirstDose', 'SecondDose', 'NumberDosesReceived', 'Resto']].plot(figsize=(15,5))
axes.legend(loc="upper left")
plt.show()


In [ ]:
dft.tail(2)


In [ ]:
x = dft.groupby('Vaccine').sum()
x['stock'] = x['NumberDosesReceived'] - x['FirstDose'] - x['SecondDose'] - x['UnknownDose']
#x = x.cumsum().tail(1)
x['perc'] = 100 * x['stock'] / x['NumberDosesReceived']
x


In [ ]:
#recebidas=2.983.590 stock=278.957 az=127.624 34% mod=101.751 (17%) pf=49.582 2.5% 


In [ ]:
dfpt[dfpt['Vaccine'] == "JANSS"]


In [ ]:
data


In [ ]:
# paises
# array(['BG', 'CY', 'CZ', 'DE', 'EE', 'EL', 'FI', 'FR', 'HR', 'HU', 'IE',
#       'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'NO', 'PL', 'PT', 'SE', 'SI',
#       'AT', 'BE', 'DK', 'ES', 'IS', 'SK', 'RO', 'LI'], dtype=object)
#data['ReportingCountry'].unique()
df = data[ (data['NumberDosesReceived'] > 0) & (data['TargetGroup'] == 'ALL') ]
df = df.copy()

#df.loc[df["ReportingCountry"] == "SE", ['NumberDosesReceived','FirstDose','SecondDose']].sum()

doses = df.groupby("ReportingCountry")[['NumberDosesReceived','FirstDose','SecondDose']].sum().reset_index(level=0)
pop = df[['ReportingCountry', 'Population']].drop_duplicates()


df = doses.merge(pop, how="left", on="ReportingCountry")

df['perc_doses'] = 100 * df['NumberDosesReceived'] / df['Population']
df['perc_doses1'] = 100 * df['FirstDose'] / df['Population']
df['perc_doses2'] = 100 * df['SecondDose'] / df['Population']

df.sort_values(['perc_doses'], ascending=False).reset_index(drop=True)


In [ ]:
data['Region'].unique()


In [ ]:
dfr = dfpt[dfpt.NumberDosesReceived > 0].copy()
dfr.set_index("date", inplace=True)

fig, axes = plt.subplots(figsize=(15,5))
for vac in VACCINES:
    axes.plot(dfr[dfr["Vaccine"] == vac].NumberDosesReceived, label=VACCINES[vac], lw=4, marker='o', markersize=8, alpha=1) # , color=C[v][0])
axes.plot(dfr.groupby(["date"]).sum()["NumberDosesReceived"], label="Total", lw=4, marker='o', markersize=8, alpha=1)

axes.legend(loc="upper left")
plt.title("Vacinas Recebidas por semana")
plt.xticks(list(reversed(list(reversed(list(set(list(dfr.index))))))), rotation=90)
plt.ylim(0)
plt.tight_layout()
if SAVE_IMAGES: plt.savefig(f"../../temp/vacinas_ecdc_recebidas.png")
plt.show()


In [ ]:
dfr.groupby(["date"]).sum()["NumberDosesReceived"] #.plot(figsize=(15,5))


In [ ]:
dfr.groupby(["date", "Vaccine"]).sum()["NumberDosesReceived"] #.plot(figsize=(15,5))


In [ ]:
data[ data.FirstDoseRefused > 0].groupby( ["ReportingCountry", "TargetGroup"] ).sum()


In [ ]:
for v in VACCINES:
    temp = dfpt[dfpt['Vaccine'] == v].groupby(['date', 'TargetGroup']).sum()
    dfv = pd.pivot_table(temp, values='FirstDose', index="date", columns="TargetGroup").reset_index()
    print(f"Vaccine {v}")
    print(dfv)


In [ ]:
dfrecv = dfpt[ dfpt.NumberDosesReceived > 0] # [['date', 'Vaccine', 'NumberDosesReceived']] # .groupby(['date', 'Vaccine']) # .cumsum()
dfrecv['resto'] = dfrecv['NumberDosesReceived'] - dfrecv['FirstDose'] - dfrecv['SecondDose']
for vac in VACCINES:
    temp = dfrecv[ dfrecv.Vaccine == vac ]
    temp['resto_sum'] = temp['resto'].cumsum()
    temp.set_index('date', inplace=True)
    print(temp[['Vaccine', 'resto_sum']] .tail())


In [ ]:
fig, axes = plt.subplots(figsize=(15,5))
k, kk = 'resto_sum', 'Stock'
idx = set()
pd.set_option('display.expand_frame_repr', False)

for v in VACCINES:
    temp = dfrecv[ dfrecv.Vaccine == v ].copy()
    if v == 'AZ':
        # https://twitter.com/Mmacedo_tweets/status/1415397265343143937
        temp.loc[ temp.date == '2021-07-03', 'resto'] -= 178 * 1000
        # https://twitter.com/mmacedo_tweets/status/1428251190144425987
        temp.loc[ temp.date == '2021-08-14', 'resto'] -= 185 * 1000
        # https://www.publico.pt/2021/08/27/sociedade/noticia/portugal-enviou-553-mil-doses-vacinas-3-milhoes-prometidos-palop-timorleste-1975381
        temp.loc[ temp.date == '2021-05-15', 'resto'] -= 24 * 1000
        temp.loc[ temp.date == '2021-07-24', 'resto'] -= 24 * 1000
        temp.loc[ temp.date == '2021-08-21', 'resto'] -= (553 - 24 - 24 - 185) * 1000 
    temp[f'resto_sum_{v}'] = temp['resto'].cumsum()
    temp.set_index('date', inplace=True)
    idx.update(temp.index)
    if v == 'AZ':
        print(temp[['FirstDose', 'SecondDose', 'UnknownDose', 'NumberDosesReceived', 'Vaccine', 'resto', f'resto_sum_{v}']].tail(60))

    axes.plot(temp[f'resto_sum_{v}'], label=f"{VACCINES[v]} {kk}", lw=4, marker='o', markersize=8, alpha=1) # , color=C[v][0])

plt.xticks(list(reversed(list(reversed(list(set(list(idx)))))[0::1])), rotation=90)
#axes.axvline(linewidth=2, ls="--", color='white', alpha=0.75, x=datetime.datetime.today())
axes.legend(loc="upper left")
plt.title("Stock existente por semana")
plt.ylim(0)
plt.tight_layout()
if SAVE_IMAGES: plt.savefig("../../temp/vacinas_ecdc_stock.png")
plt.show()



In [ ]:
dfrecv[dfrecv.Vaccine == 'JANSS']


In [ ]:
last_date = dfrecv.date.unique()
last_date.sort()
last_date = last_date[-1]
dfrecv[ dfrecv.date == last_date ]


In [ ]:
dfr = dfpt[ (dfpt.Vaccine == 'COM') & (dfpt.TargetGroup != 'ALL') ].copy()
dfr.set_index("date", inplace=True)
ages = list(dfr.TargetGroup.unique())
# ages.remove('Age<18')
ages.remove('AgeUNK')

fig, axes = plt.subplots(figsize=(15,5))
i=0
for age in ages:
    axes.plot(dfr[dfr["TargetGroup"] == age].FirstDose, label=f"{age} 1ª", lw=4, marker='o', markersize=8, alpha=1) # , color=C[i])
    axes.plot(dfr[dfr["TargetGroup"] == age].SecondDose, label=f"{age} 2ª", lw=2, marker='o', markersize=8, alpha=0.6, color=plt.gca().lines[-1].get_color() )
#axes.plot(dfr.groupby(["date"]).sum()["NumberDosesReceived"], label="Total", lw=4, marker='o', markersize=8, alpha=1)

axes.legend(loc="upper left")
plt.xticks(list(reversed(list(reversed(list(set(list(dfr.index))))))), rotation=90)
plt.ylim(0)
plt.tight_layout()
if SAVE_IMAGES: plt.savefig(f"../../temp/vacinas_ecdc_az.png")
plt.show()
dfr.groupby(['TargetGroup']).sum()
